In [42]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime  # Change the import statement
from datetime import timedelta

# Define global variables for dataframes
student_df = None
engagement_df = None
ums_att_df = None
ums_avg_att_df = None

ums_upload_avg_att=None

# Edoofy app information
edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

# UMS app information
ums_base_url = "https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}

# Asynchronous function to fetch data from a table
async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df

def map_students_to_engagement(student_df, engagement_df):
    # Ensure that '_id' and 'KAM-group-name' are in student_df
    if '_id' in student_df.columns and 'KAM-group-name' in student_df.columns:
        # Create a mapping dictionary from student_df
        mapping_dict = dict(zip(student_df['_id'], student_df['KAM-group-name']))

        # Map the 'student' column in engagement_df to 'KAM-group-name' using the mapping_dict
        engagement_df['student'] = engagement_df['student'].map(mapping_dict)
    else:
        print("Error: '_id' or 'KAM-group-name' not in student_df columns")

    return engagement_df



async def main():
    global student_df, engagement_this_month_df, engagement_previous_months_df, ums_att_df, ums_avg_att_df
    
    async with aiohttp.ClientSession() as session:
        # Fetch Student table from Edoofy without constraints
        student_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student")

        # Calculate the start and end date for the last 12 months (excluding this month)
        current_date = datetime.now()
        end_date = current_date.replace(day=1) - timedelta(days=1)
        start_date = end_date - timedelta(days=365)

        # Create constraints for engagement dataframe for this month
        this_month_start = current_date.replace(day=1)
        this_month_end = current_date
        engagement_this_month_constraints = [
            {'key': 'engagement-date', 'constraint_type': 'greater than', 'value': this_month_start.isoformat()},
            {'key': 'engagement-date', 'constraint_type': 'less than', 'value': this_month_end.isoformat()},
            {'key': 'student', 'constraint_type': 'equals', 'value': '1695736497533x818400363201798900'}
        ]

        # Create constraints for engagement dataframe for previous months
        engagement_previous_months_constraints = [
            {'key': 'engagement-date', 'constraint_type': 'greater than', 'value': start_date.isoformat()},
            {'key': 'engagement-date', 'constraint_type': 'less than', 'value': this_month_start.isoformat()},
            {'key': 'student', 'constraint_type': 'equals', 'value': '1695736497533x818400363201798900'}
        ]

        # Fetch Engagement table from Edoofy with constraints for this month
        engagement_this_month_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", constraints=engagement_this_month_constraints)

        # Fetch Engagement table from Edoofy with constraints for previous months
        engagement_previous_months_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", constraints=engagement_previous_months_constraints)

        # Create a mapping dictionary from student_df for '_id' to 'KAM-group-name'
        student_id_to_KAM = dict(zip(student_df['_id'], student_df['KAM-group-name']))
        student_id_to_EWYL = dict(zip(student_df['_id'], student_df['EWYL-group-name']))
        
        # Apply mapping to 'student' column in engagement dataframes for EWYL-group-name
        try:
            engagement_this_month_df['ewyl'] = engagement_this_month_df['student'].map(student_id_to_EWYL)
            engagement_previous_months_df['ewyl'] = engagement_previous_months_df['student'].map(student_id_to_EWYL)
        except KeyError:
            print("KeyError: 'student' column not found in engagement dataframes, proceeding without adding 'ewyl' column.")

        # Apply mapping to 'student' column in engagement dataframes for KAM-group-name
        try:
            engagement_this_month_df['student'] = engagement_this_month_df['student'].map(student_id_to_KAM)
            engagement_previous_months_df['student'] = engagement_previous_months_df['student'].map(student_id_to_KAM)
        except KeyError:
            print("KeyError: 'student' column not found in engagement dataframes, proceeding without mapping.")

        
        # Fetch tables from UMS without constraints
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")
        ums_avg_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")

#function to sort dates in dataframes.
def sort_dataframe_by_date(df, date_column):
    # Convert the date column to datetime format if it's not already
    df[date_column] = pd.to_datetime(df[date_column])

    # Sort the dataframe in descending order by the date column
    sorted_df = df.sort_values(by=date_column, ascending=False)

    return sorted_df

#function to process the fetched data
def create_processed_ums_att(ums_att_df, engagement_this_month_df, engagement_previous_months_df):
    columns = ['activity-lesson', 'admissions-group-name', 'attendance-type', 
               'ewyl-group-name', 'present', 'date']
    processed_ums_att = pd.DataFrame(columns=columns)

    # Combining unique students from both UMS attendance and engagement dataframes
    unique_students_ums = set(ums_att_df['ewyl-group-name'].unique())
    
    unique_students_engagement = set(engagement_this_month_df['ewyl'].unique()).union(
                                 set(engagement_previous_months_df['ewyl'].unique()))
    all_unique_students = unique_students_ums.union(unique_students_engagement)
    

    for student in all_unique_students:
        # Handling students in UMS attendance dataframe
        if student in unique_students_ums:
            student_dates = ums_att_df[ums_att_df['ewyl-group-name'] == student]['date'].sort_values()
            
            for date in student_dates:
                if (engagement_this_month_df[(engagement_this_month_df['ewyl'] == student) & 
                    (engagement_this_month_df['engagement-date'] == date)].empty and
                    engagement_previous_months_df[(engagement_previous_months_df['ewyl'] == student) & 
                    (engagement_previous_months_df['engagement-date'] == date)].empty):

                    missing_dates = student_dates[student_dates >= date]
                    for missing_date in missing_dates:
                        ums_record = ums_att_df[(ums_att_df['ewyl-group-name'] == student) & 
                                                (ums_att_df['date'] == missing_date)].iloc[0]
                        new_row = pd.DataFrame([{
                            'activity-lesson': ums_record['activity-lesson'],
                            'admissions-group-name': ums_record['admissions-group-name'],
                            'attendance-type': ums_record['attendance-type'],
                            'ewyl-group-name': student,
                            'present': ums_record['present'],
                            'date': missing_date
                        }], columns=columns)
                        processed_ums_att = pd.concat([processed_ums_att, new_row], ignore_index=True)
                    break  # Once missing dates are added, proceed to next student

        # Handling students present in engagement dataframes but not in UMS attendance
        elif student not in unique_students_ums:
            # Logic for students present in engagement but not in UMS
            # Assuming 'activity-lesson' and 'attendance-type' take the most recent values from engagement dataframes
            # The date is assumed to be the most recent date from the engagement dataframes
            most_recent_engagement_record = pd.concat([engagement_this_month_df[engagement_this_month_df['ewyl'] == student], 
                                                       engagement_previous_months_df[engagement_previous_months_df['ewyl'] == student]]).sort_values(by='engagement-date', ascending=False).iloc[0]
            
            new_row = pd.DataFrame([{
                'activity-lesson': most_recent_engagement_record['engagement-type'],
                'admissions-group-name': student,
                'attendance-type': most_recent_engagement_record['engagement-type'],
                'ewyl-group-name': student,
                'present': False,  # Assuming absent if not in UMS
                'date': most_recent_engagement_record['engagement-date']
            }], columns=columns)
            processed_ums_att = pd.concat([processed_ums_att, new_row], ignore_index=True)

    return processed_ums_att

# this_month_df
# last_month_df
# If a row for thhis month already exists then get the row ID 
#  call process_avg_att(this_month_df, true) and get the updated value for the attendance % for this month
#  update that row in atendance sumary with that new value for att%thismonth
# elif 
#  create a df with one row for this month (attendance summary table)
#  populate all columns in the df apart from the % ones
#  call process_avg_att(this_month_df, true) to get the value for att%thismonth
#  call process_avg_att(last_month_df, false) to get the value for att%tilllastmonth

def get_percentage(number_of_present, total_session):
    percent = (number_of_present/total_session) * 100
    return percent

def process_avg_att(processed_ums_att, thisOrLastMonth):
    columns = ['admissions-group-name', 'attendance-percentage', 'avg-att-percent-till-last-month',
               'ewyl-group-name', 'first-day-of-month', 'month','year']
    this_month_eng = pd.DataFrame(columns=columns)    
    count_yes=0
    count_all=0
    
    for student in processed_ums_att
        if processed_ums_att['present'] == True
            count_yes+=1
        count_all+=1
    percent = get_percentage(count_yes, count_all)
    if
    process_avg_att['avg-att-percent-till-last-month'] = get_percentage(count_yes, count_all)
        
    
await main()


'''
print(student_df.head())  # Access Student dataframe
print(engagement_this_month_df.head())  # Access Engagement dataframe for this month
print(engagement_previous_months_df.head())  # Access Engagement dataframe for previous months
print(ums_att_df.head())  # Access UMS Attendance dataframe
print(ums_avg_att_df.head())  # Access UMS Average Attendance dataframe
'''

engagement_this_month_df = engagement_this_month_df[engagement_this_month_df['engagement-type'].isin(['IE Call', 'IE Chat', 'Activity', 'Lesson'])]
engagement_previous_months_df = engagement_previous_months_df[engagement_previous_months_df['engagement-type'].isin(['IE Call', 'IE Chat', 'Activity', 'Lesson'])]

engagement_this_month_df = sort_dataframe_by_date(engagement_this_month_df, 'engagement-date')
engagement_previous_months_df = sort_dataframe_by_date(engagement_previous_months_df, 'engagement-date')

ums_att_df = sort_dataframe_by_date(ums_att_df, 'date')

processed_ums_att = create_processed_ums_att(ums_att_df, engagement_this_month_df, engagement_previous_months_df)



SyntaxError: expected ':' (3529132192.py, line 229)

In [40]:
engagement_this_month_df.to_csv("engagement_this_month.csv",index=False)
ums_att_df.to_csv("ums_att.csv",index=False) 
engagement_previous_months_df.to_csv("engagement_previous_months_df.csv",index=False)

In [17]:
print(pd.__version__)

2.1.2


In [41]:
processed_ums_att.to_csv("processed_ums_att.csv",index=False) 


In [29]:
print(student_dates)

NameError: name 'student_dates' is not defined